# Modèle AF (via PyAEZ)

On utilise la bibliothèque PyAEZ, open source de la FAO, qui fonctionne comme suit :
![title](images/pyaez_info.png)

On commence par inialiser la partie agro-climatique en utilisant la latitude minimale et la longitude maimale de la parecelle, supposée rectangulaire (possibilité de mettre un masque sinon). 

On utilise le module python soilgrids et ? pour réunir les informations sur la parcelle.

In [7]:
#imports de modules
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys
import pyaez.ClimateRegime as ClimateRegime
import pyaez.CropSimulation as CropSimulation
import pyaez.UtilitiesCalc as UtilitiesCalc
import pyaez.SoilConstraints as SoilConstraints
import pyaez.ClimaticConstraints as ClimaticConstraints

In [8]:
def params_geo(lat_min,lat_max,freq):
    """recupere les données geo obtenues via soilgrids et ? selon la parcelle
    A CODER
    """
    return elevation, min_temp, rel_humidity, short_rad, wind_speed

def init_params_geo(lat_min,lat_max,freq='daily'):
    """initialise les valeurs du modèle geo selon la localisation de la parcelle et la fréquence 
    des données géo
    NB. Il est possible d'appliquer un masque sur la zone géographique pour des parcelles plus biscornues
    """
    #initialisation du modèle climatique
    clim_reg = ClimateRegime.ClimateRegime()
    #3d numpy (x,y,t) des différents paramètres géo du modèle, via soilgrids
    elevation, min_temp, rel_humidity, short_rad, wind_speed = params_geo(lat_min,lat_max,freq)
    #masques anti valeurs aberrantes
    rel_humidity[rel_humidity<0] = 0
    short_rad[short_rad<0]=0
    wind_speed[wind_speed<0]=0
    #réglage des params long, lat et elevation
    clim_reg.setLocationTerrainData(lat_min, lat_max, elevation)
    #réglage des autres params selon fréquences des données (journalières ou par mois)
    if freq == 'daily':
        clim_reg.setDailyClimateData(min_temp, max_temp, precipitation, short_rad,wind_speed, rel_humidity)
    else: #freq == 'monthly', moins précis
        clim_reg.setMonthlyClimateData(min_temp, max_temp, precipitation, short_rad,wind_speed, rel_humidity)
    return clim_reg

def init_params_aez(lat_min,lat_max,freq='daily'):
    """initialise les valeurs du modèle agri selon la localisation de la parcelle et la fréquence 
    des données géo
    NB. Il est possible d'appliquer un masque sur la zone géographique pour des parcelles plus biscornues
    """
    #initialisation du modèle climatique
    aez = CropSimulation.CropSimulation()
    #3d numpy (x,y,t) des différents paramètres géo du modèle, via soilgrids
    elevation, min_temp, rel_humidity, short_rad, wind_speed = params_geo(lat_min,lat_max,freq)
    #masques anti valeurs aberrantes
    rel_humidity[rel_humidity<0] = 0
    short_rad[short_rad<0]=0
    wind_speed[wind_speed<0]=0
    #réglage des params long, lat et elevation
    aez.setLocationTerrainData(lat_min, lat_max, elevation)
    #réglage des autres params selon fréquences des données (journalières ou par mois)
    if freq == 'daily':
        aez.setDailyClimateData(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
    else: #freq == 'monthly', moins précis
        aez.setMonthlyClimateData(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
    return aez

def run_aez(aez, clim_reg):
    """execute l'évolution de la parcelle (version pluie et version irriguée), renvoyant les rendements
    Nombreux paramètres à régler, on se contente ici du minimum
    """
    tclimate = clim_reg.getThermalClimate()
    #utilisation d'un output du modèle geo pour input de modele agri
    aez.setThermalClimateScreening(tclimate, no_t_climate=[2, 5, 6]) #no_t_climate = climats défavorables à la culture
    #parametres du modele de culture ###### a comprendre et choisir, liés aux cultures #####
    aez.setCropParameters(LAI=2.3, HI=0.33, legume=0, adaptability=4, cycle_len=115, D1=0.3, D2=1)
    aez.setCropCycleParameters(stage_per=[16, 26, 33, 25], kc=[0.3, 1.2, 0.5], kc_all=0.85, yloss_f=[0.4, 0.4, 0.9, 0.5], yloss_f_all=1.25)
    aez.setSoilWaterParameters(Sa=100, pc=0.5)
    #execution du modele (peut etre long askip)
    aez.simulateCropCycle( start_doy=1, end_doy=365, step_doy=1, leap_year=False) #resultats en kg / hectare

def contraintes(clim_reg,aez,irr='R'):
    """prise en compte des contraintes climatiques, de sol et de terrain
    A FINIR
    """
    #module utile
    obj_utilities = UtilitiesCalc.UtilitiesCalc()
    
    #contraintes climatiques
    clim_constraints = ClimaticConstraints.ClimaticConstraints()
    #recuperation de données geo
    lgp = clim_reg.getLGP(Sa = 100)
    lgp_class = clim_reg.getLGPClassified(lgp)
    lgp_equv = clim_reg.getLGPEquivalent()
    #pluie (Rain)
    yield_map_rain = aez.getEstimatedYieldRainfed() 
    clim_adj_yield_rain = clim_constraints.applyClimaticConstraints(lgp_equv, yield_rain, 'R')
    clim_adj_yield_rain_class = obj_utilities.classifyFinalYield(clim_adj_yield_irr)
    #irrigée
    yield_map_irr = aez.getEstimatedYieldIrrigated()
    clim_adj_yield_irr = clim_constraints.applyClimaticConstraints(lgp_equv, yield_irr, 'I') 
    clim_adj_yield_irr_class = obj_utilities.classifyFinalYield(clim_adj_yield_irr)
    
    #contraintes du sol
    soil_constraints = SoilConstraints.SoilConstraints()

    #contraintes du terrain
    terrain_constraints = TerrainConstraints.TerrainConstraints()


Le code est à finir mais pourrait permettre de faire tourner un modèle relativmeent réaliste. Il n'est cependant pas vraiment pensé pour de l'agroforesterie. Il faudrait réussir à l'adapter, je ne sais pas dans quelle mesure c'est réalisable. Mais l'avantage c'est que tout est open source et pas hyper obscur. 

Reste des trucs à coder notamment sur les jonctions avec les api.